<a href="https://colab.research.google.com/github/GaCampioto/tech-challenges-fiap-AI-for-developers/blob/main/tech_challenge_3_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine tuning

Após formatar os dados vamos começar o processo de fine tuning.

Este notebook foi executado em uma A100 RAM alta

In [2]:
# habilitando a utilização dos arquivos do drive como fonte de dados
from google.colab import drive
drive.mount('/content/drive')

tc3_path = '/content/drive/MyDrive/Colab Notebooks/tech-challenge-3'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Instalação dependências, importando pacotes e definindo variaveis

In [ ]:
# alterando a versão do torch do colab devido a problemas com unsloth e xformers
!pip uninstall torch -y
!pip install torch==2.4.0

import torch
torch.__version__

Found existing installation: torch 2.4.0+cu121
Uninstalling torch-2.4.0+cu121:
  Successfully uninstalled torch-2.4.0+cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 3.6 MB/s eta 0:00:00


'2.4.0+cu121'

In [ ]:
# instalação do Unsloth, Xformers e outros pacotes necessários
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes tritonserver
!pip install transformers datasets
!pip install xformers
!pip install trl
!pip install peft
!pip install accelerate
!pip install bitsandbytes
!pip install tritonserver

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ey4ijx5k/unsloth_d8b195ad6f5442f3b9861f5f167d9928
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ey4ijx5k/unsloth_d8b195ad6f5442f3b9861f5f167d9928
  Resolved https://github.com/unslothai/unsloth.git to commit d91d40a7b6b556f2d1fdd3e1e430f7a76a799627
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 47.6 MB/s eta 0:00:00
   

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

max_seq_length = 2048
dtype = None
load_in_4bit = True

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


## Processamento dos dados

O próximo passo é gerar um método que irá ler nossos dados preparados na etapa anterior e transforma-los em uma entrada para o modelo.

In [ ]:
def format_dataset_into_model_input(df, index):
    def extract_variables(text):
      titleStart = text.find("[|Title|]") + len("[|Title|]")
      titelEnd = text.find("[|eTitle|]")
      contentStart = text.find("[|content|]") + len("[|content|]")
      contentEnd = text.find("[|eContent|]")

      instruction = text.split( '\n')[0]
      input_text = text[titleStart:titelEnd].strip()
      output_text = text[contentStart:contentEnd].strip()

      return instruction, input_text, output_text

    instructions = []
    inputs = []
    outputs = []

    for prompt in df:
      instruction, input_text, output_text = extract_variables(prompt['input'])
      instructions.append(instruction)
      inputs.append(input_text)
      outputs.append(output_text)

    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    with open(f"{tc3_path}/formatted-data-{index}.json", 'w', encoding='utf-8') as output_file:
        json.dump(formatted_data, output_file, ensure_ascii=False, indent=4)

    print(f"Dataset salvo em '{tc3_path}/formatted-data-{index}.json'")

In [ ]:
train_dfs = []

for i in range(1, 11):
  with open(f"{tc3_path}/trn_prepared_{i}.json", 'r') as input_file:
    json_data = json.load(input_file)
    train_dfs.append(json_data)

index = 1
for df in train_dfs:
  format_dataset_into_model_input(df, index)
  index += 1

Dataset salvo em '/content/drive/MyDrive/Colab Notebooks/tech-challenge-3/formatted-data-1.json'
Dataset salvo em '/content/drive/MyDrive/Colab Notebooks/tech-challenge-3/formatted-data-2.json'
Dataset salvo em '/content/drive/MyDrive/Colab Notebooks/tech-challenge-3/formatted-data-3.json'
Dataset salvo em '/content/drive/MyDrive/Colab Notebooks/tech-challenge-3/formatted-data-4.json'
Dataset salvo em '/content/drive/MyDrive/Colab Notebooks/tech-challenge-3/formatted-data-5.json'
Dataset salvo em '/content/drive/MyDrive/Colab Notebooks/tech-challenge-3/formatted-data-6.json'
Dataset salvo em '/content/drive/MyDrive/Colab Notebooks/tech-challenge-3/formatted-data-7.json'
Dataset salvo em '/content/drive/MyDrive/Colab Notebooks/tech-challenge-3/formatted-data-8.json'
Dataset salvo em '/content/drive/MyDrive/Colab Notebooks/tech-challenge-3/formatted-data-9.json'
Dataset salvo em '/content/drive/MyDrive/Colab Notebooks/tech-challenge-3/formatted-data-10.json'


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # recuperando um modelo pré treinado llama 3
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit, # load_in_4_bit verdadeiro para que tenhamos melhor performance com a memória
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-

Agora precisamos criar o modelo

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Agora é necessário criar os prompts que serão enviados para executar o treinamento

In [ ]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

# recuperando o primeiro chunk de dados formatados
train_ds = load_dataset("json", data_files=f"{tc3_path}/formatted-data-1.json", split='train')
train_ds = train_ds.map(formatting_prompts_func, batched = True)

print(train_ds)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/139041 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 139041
})


## Teste pré treinamento

In [ ]:
def execute_inference(model, tokenizer, prompt, item):
  FastLanguageModel.for_inference(model)
  inputs = tokenizer(
  [
  prompt.format(
          item['instruction'],
          item['input'],
          "",
      )
  ], return_tensors = "pt").to("cuda")

  from transformers import TextStreamer
  text_streamer = TextStreamer(tokenizer)
  _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)


prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

execute_inference(model, tokenizer, prompt, train_ds[0])
execute_inference(model, tokenizer, prompt, train_ds[1])
execute_inference(model, tokenizer, prompt, train_ds[2])

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a content for the title above.

### Input:
Girls Ballet Tutu Neon Pink

### Response:
This is a request to write a content for the title above. The input provides further context. The response should be appropriate and relevant to the request.
<|end_of_text|>
<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a content for the title above.

### Input:
Mog's Kittens

### Response:
Mog's Kittens

### Instruction:
Write a content for the title above.

### Input:
Mog's Kittens

### Response:
Mog's Kittens

### Instruction:
Write a content for the title above.

### Input:
Mog's Kittens

### Response:
Mog's Kittens

### Instruction:
Write a content fo

Mesmo sem treinamento é possível ver que existe uma inteligência por trás da geração de descrições, mas o modelo parece não respeitar o que pedido.

## Treinamento

Por fim, vamos instanciar e executar o treinamento

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # testei com 10, 20, 30 e 60, sendo que o resultado foi melhor com 60.
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/139041 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 139,041 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.592200
2,3.585800
3,3.739000
4,3.894900
5,3.304700
6,3.649900
7,3.170200
8,3.388300
9,2.910800
10,2.825000


TrainOutput(global_step=60, training_loss=2.20359646876653, metrics={'train_runtime': 82.6793, 'train_samples_per_second': 5.806, 'train_steps_per_second': 0.726, 'total_flos': 8295788676268032.0, 'train_loss': 2.20359646876653, 'epoch': 0.003452194301002575})

## Salvando o modelo para utilização no notebook de steps

Como último passo do processo de treinamento, vamos salvar o modelo treinado para que seja possível treinar com os outros chunks de dados.

In [ ]:
model.save_pretrained(f"{tc3_path}/lora_model")
tokenizer.save_pretrained(f"{tc3_path}/lora_model")

('/content/drive/MyDrive/Colab Notebooks/tech-challenge-3/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/tech-challenge-3/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/tech-challenge-3/lora_model/tokenizer.json')

## Execução do modelo após o treinamento

Após executar o treinamento com todos os chunks, voltamos para essa etapa na qual vamos carregar o modelo pré treinado e salvo no google drive, para que possamos executar os mesmos testes feitos no pré treinamento.

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
  model_name = f"{tc3_path}/lora_model",
  max_seq_length = max_seq_length,
  dtype = dtype,
  load_in_4bit = load_in_4bit,
)

FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

In [ ]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

execute_inference(model, tokenizer, prompt, train_ds[0])
execute_inference(model, tokenizer, prompt, train_ds[1])
execute_inference(model, tokenizer, prompt, train_ds[2])

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a content for the title above.

### Input:
Girls Ballet Tutu Neon Pink

### Response:
The tutu is a classical dance skirt. It is made from many layers of net or tulle, supported by a stiffened waistband and worn over pointe shoes or ballet slippers. The tutu is a signature feature of classical ballet and is a symbol of the art form.<|end_of_text|>
<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a content for the title above.

### Input:
Mog's Kittens

### Response:
Mog's Kittens is a charming story of a cat and her kittens.  The story is a simple, yet a very sweet story about a cat named Mog and her kittens.  The story is told in a series of 

Com essa execução após o treinamento do modelo é possível concluir que:

1.   O modelo aprendeu a gerar o campo `content` com base no `title`
2.   O modelo passou a respeitar o formato de saída, entendendo que só deveriam existir 3 campos.

Também é possível verificar que durante os treinamento o loss caiu consideravelmente, começando em 3.59 e terminando (no outro notebook) na casa dos 1.5, tendo o menor valor em 1.1. Acredito que o loss está alto, mas confesso que não sei o que poderia melhorar.